In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"D:\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [3]:
from sqlalchemy import text
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
import json
import zlib
import base64
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

In [6]:
silnik = utworz_engine_do_db()

In [18]:
q_select = text("""
WITH ile_razy_wystepuje AS (
    SELECT
        MISJA_ID_MOJE_PK,
        MISJA_ID_Z_GRY,
        MISJA_URL_WIKI,
        COUNT(MISJA_ID_Z_GRY) OVER (PARTITION BY MISJA_ID_Z_GRY) AS CNT
    FROM dbo.MISJE
    WHERE MISJA_ID_Z_GRY != 123456789
),

cnt_i_dlugosc AS (
    SELECT
        MISJA_ID_MOJE_PK,
        MISJA_ID_Z_GRY,
        MISJA_URL_WIKI,
        CNT,
        LEN(MISJA_URL_WIKI) AS DLUGOSC_URL,
        MIN(LEN(MISJA_URL_WIKI)) OVER (PARTITION BY MISJA_ID_Z_GRY) AS MIN_DLUGOSC
    FROM ile_razy_wystepuje
    WHERE CNT > 1
)

SELECT
    MISJA_ID_MOJE_PK,
    CASE
        WHEN CNT = 2 AND DLUGOSC_URL = MIN_DLUGOSC THEN '123456789'
        WHEN CNT = 2 THEN MISJA_ID_Z_GRY
        WHEN CNT >= 3 AND DLUGOSC_URL = MIN_DLUGOSC THEN MISJA_ID_Z_GRY
        WHEN CNT >= 3 THEN '123456789'
        ELSE MISJA_ID_Z_GRY
    END AS MISJA_ID_Z_GRY
FROM cnt_i_dlugosc
""")

q_update = text("""
    UPDATE dbo.MISJE
    SET MISJA_ID_Z_GRY = :MISJA_ID_Z_GRY
    WHERE MISJA_ID_MOJE_PK = :MISJA_ID_MOJE_PK
""")

In [21]:
with silnik.begin() as conn:
    w = conn.execute(q_select).mappings().all()
    if len(w) > 0:
        conn.execute(q_update, w)
    else:
        print("Brak danych do dodania.")

Brak danych do dodania.


In [ ]:
def popraw_misje_z_gry(silnik):
    q_select = text("""
    WITH ile_razy_wystepuje AS (
        SELECT
            MISJA_ID_MOJE_PK,
            MISJA_ID_Z_GRY,
            MISJA_URL_WIKI,
            COUNT(MISJA_ID_Z_GRY) OVER (PARTITION BY MISJA_ID_Z_GRY) AS CNT
        FROM dbo.MISJE
        WHERE MISJA_ID_Z_GRY != 123456789
    ),

    cnt_i_dlugosc AS (
        SELECT
            MISJA_ID_MOJE_PK,
            MISJA_ID_Z_GRY,
            MISJA_URL_WIKI,
            CNT,
            LEN(MISJA_URL_WIKI) AS DLUGOSC_URL,
            MIN(LEN(MISJA_URL_WIKI)) OVER (PARTITION BY MISJA_ID_Z_GRY) AS MIN_DLUGOSC
        FROM ile_razy_wystepuje
        WHERE CNT > 1
    )

    SELECT
        MISJA_ID_MOJE_PK,
        CASE
            WHEN CNT = 2 AND DLUGOSC_URL = MIN_DLUGOSC THEN '123456789'
            WHEN CNT = 2 THEN MISJA_ID_Z_GRY
            WHEN CNT >= 3 AND DLUGOSC_URL = MIN_DLUGOSC THEN MISJA_ID_Z_GRY
            WHEN CNT >= 3 THEN '123456789'
            ELSE MISJA_ID_Z_GRY
        END AS MISJA_ID_Z_GRY
    FROM cnt_i_dlugosc
    """)

    q_update = text("""
        UPDATE dbo.MISJE
        SET MISJA_ID_Z_GRY = :MISJA_ID_Z_GRY
        WHERE MISJA_ID_MOJE_PK = :MISJA_ID_MOJE_PK
    """)

    with silnik.begin() as conn:
        w = conn.execute(q_select).mappings().all()
        if len(w) > 0:
            wynik = conn.execute(q_update, w)
            print(f"Zaktualizowano: {wynik.rowcount} wierszy.")
        else:
            print("Brak danych do dodania.")